# BigQuery Magic Commands and DML

The examples in this notebook introduce features of [BigQuery Standard SQL](https://cloud.google.com/bigquery/sql-reference/) and [BigQuery SQL Data Manipulation Language (beta)](https://cloud.google.com/bigquery/sql-reference/dml-syntax). BigQuery Standard SQL is compliant with the SQL 2011 standard. You've already seen the use of the magic command `%%bq` in the [Hello BigQuery](./Hello BigQuery.ipynb) and [BigQuery Commands](./BigQuery Commands.ipynb) notebooks. This command and others in the Google Cloud Datalab API support BigQuery Standard SQL.

## Using the BigQuery Magic command with Standard SQL

First, we will cover some more uses of the `%%bq` magic command. Let's define a query to work with:

In [3]:
%load_ext google.datalab.kernel

In [4]:
%%bq query --name UniqueNames2013
WITH UniqueNames2013 AS
(SELECT DISTINCT name
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE Year = 2013)
SELECT * FROM UniqueNames2013

Now let's list all available commands to work with `%%bq`

In [5]:
%%bq -h

usage: %bq [-h]
           {datasets,tables,query,execute,extract,sample,dryrun,udf,datasource,load,pipeline}
           ...

Execute various BigQuery-related operations. Use "%bq <command> -h" for help
on a specific command.

positional arguments:
  {datasets,tables,query,execute,extract,sample,dryrun,udf,datasource,load,pipeline}
                        commands
    datasets            Operations on BigQuery datasets
    tables              Operations on BigQuery tables
    query               Create or execute a BigQuery SQL query object,
                        optionally using other SQL objects, UDFs, or external
                        datasources. If a query name is not specified, the
                        query is executed.
    execute             Execute a BigQuery SQL query and optionally send the
                        results to a named table. The cell can optionally
                        contain arguments for expanding variables in the
                        query.
 

The `dryrun` argument in ``%%bq`` can be helpful to confirm the syntax of the SQL query. Instead of executing the query, it will only return some statistics:

In [6]:
%%bq dryrun -q UniqueNames2013

Now, let's get a small sample of the results using the `sample` argument in ``%%bq``:

In [7]:
%%bq sample -q UniqueNames2013

name
Emily
Amelia
Victoria
Jade
Anna
Alyssa
Mckinley
Athena
Emery
Jordyn


Finally, We can use the `execute` command in %%bq to display the results of our query:

In [8]:
%%bq execute -q UniqueNames2013

name
Emily
Amelia
Victoria
Jade
Anna
Alyssa
Mckinley
Athena
Emery
Jordyn


# Using Google BigQuery SQL Data Manipulation Language

Below, we will demonstrate how to use Google BigQuery SQL Data Manipulation Language (DML) in Datalab.

## Preparation

First, let's import the BigQuery module, and create a sample dataset and table to help demonstrate the features of Google BigQuery DML.

In [9]:
import google.datalab.bigquery as bq

In [10]:
# Create a new dataset (this will be deleted later in the notebook)
sample_dataset = bq.Dataset('sampleDML')
if not sample_dataset.exists():
  sample_dataset.create(friendly_name = 'Sample Dataset for testing DML', description = 'Created from Sample Notebook in Google Cloud Datalab')
  sample_dataset.exists()

In [11]:
# To create a table, we need to create a schema for it.
# Its easiest to create a schema from some existing data, so this
# example demonstrates using an example object
fruit_row = {
  'name': 'string value',
  'count': 0
}

sample_table1 = bq.Table("sampleDML.fruit_basket").create(schema = bq.Schema.from_data([fruit_row]), 
                                                          overwrite = True)

## Inserting Data

We can add rows to our newly created `fruit_basket` table by using an `INSERT` statement in our BigQuery Standard SQL query.

In [12]:
%%bq query
INSERT sampleDML.fruit_basket (name, count)
VALUES('banana', 5),
      ('orange', 10),
      ('apple', 15),
      ('mango', 20)

name,count
apple,15
mango,20
orange,10
banana,5


You may rewrite the previous query as:

In [13]:
%%bq query
INSERT sampleDML.fruit_basket (name, count)
SELECT * 
FROM UNNEST([('peach', 25), ('watermelon', 30)])

name,count
apple,15
mango,20
orange,10
banana,5
watermelon,30
peach,25


You can also use a `WITH` clause with `INSERT` and `SELECT`.

In [14]:
%%bq query
INSERT sampleDML.fruit_basket(name, count)
WITH w AS (
  SELECT ARRAY<STRUCT<name string, count int64>>
      [('cherry', 35),
      ('cranberry', 40),
      ('pear', 45)] col
)
SELECT name, count FROM w, UNNEST(w.col)

name,count
apple,15
mango,20
orange,10
banana,5
watermelon,30
peach,25
cherry,35
cranberry,40
pear,45


Here is an example that copies one table's contents into another. First we will create a new table.

In [15]:
fruit_row_detailed = {
  'name': 'string value',
  'count': 0,
  'readytoeat': False
}
sample_table2 = bq.Table("sampleDML.fruit_basket_detailed").create(schema = bq.Schema.from_data([fruit_row_detailed]), 
                                                                   overwrite = True)

In [16]:
%%bq query
INSERT sampleDML.fruit_basket_detailed (name, count, readytoeat)
SELECT name, count, false
FROM sampleDML.fruit_basket

name,count,readytoeat
cherry,35,False
apple,15,False
peach,25,False
watermelon,30,False
mango,20,False
orange,10,False
banana,5,False
pear,45,False
cranberry,40,False


## Updating Data

You can update rows in the `fruit_basket` table by using an `UPDATE` statement in the BigQuery Standard SQL query. We will try to do this using the Datalab BigQuery API.

In [17]:
%%bq query
UPDATE sampleDML.fruit_basket_detailed
SET readytoeat = True
WHERE name = 'banana'

name,count,readytoeat
watermelon,30,False
peach,25,False
apple,15,False
mango,20,False
cranberry,40,False
orange,10,False
cherry,35,False
pear,45,False
banana,5,True


To view the contents of a table in BigQuery, use `%%bq tables view` command:

In [18]:
%%bq tables view --name sampleDML.fruit_basket_detailed

name,count,readytoeat
watermelon,30,False
peach,25,False
apple,15,False
mango,20,False
cranberry,40,False
orange,10,False
cherry,35,False
pear,45,False
banana,5,True


## Deleting Data

You can delete rows in the `fruit_basket` table by using a `DELETE` statement in the BigQuery Standard SQL query.

In [19]:
%%bq query
DELETE sampleDML.fruit_basket
WHERE name in ('cherry', 'cranberry')

name,count
apple,15
mango,20
orange,10
banana,5
watermelon,30
peach,25
pear,45


Use the following query to delete the corresponding entries in `sampleDML.fruit_basket_detailed`

In [20]:
%%bq query
DELETE sampleDML.fruit_basket_detailed
WHERE NOT EXISTS
  (SELECT * FROM sampleDML.fruit_basket
  WHERE fruit_basket_detailed.name = fruit_basket.name)

name,count,readytoeat
orange,10,False
apple,15,False
mango,20,False
pear,45,False
watermelon,30,False
peach,25,False
banana,5,True


## Deleting Resources

In [21]:
# Clear out sample resources
sample_dataset.delete(delete_contents = True)